## Hands-on Tutorial: Quantifying and Reducing Gender Stereotypes in Word Embeddings

Ensuring fairness in algorithmically-driven decision-making is important to avoid inadvertent cases of bias and perpetuation of harmful stereotypes. However, modern natural language processing techniques, which learn model parameters based on data, might rely on implicit biases presented in the data to make undesirable stereotypical associations. Such a danger is facing us with word embedding, a popular framework to represent text data as vectors which has been used in many machine learning and natural language processing tasks. Recent results ([1](https://arxiv.org/abs/1607.06520), [2](https://arxiv.org/abs/1608.07187)) show that even word embeddings trained on Google News articles exhibit female/male gender stereotypes to a disturbing extent. This raises concerns because of their widespread use, as we describe, often tends to amplify these biases. 

In the following, we provide step-by-step instructions to demonstrate and quanitfy the biases in word embedding.



In [1]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
# mkdir debiaswe_tutorial
# cd debiaswe_tutorial
# git clone https://github.com/tolga-b/debiaswe.git

# To reduce the time of downloading data, we provide as subset of GoogleNews-vectors in the following location:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [2]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Google News texts consisting of 3 million English words and terms. The embedding maps each word into a 300-dimension vector. 

In [3]:
# load google news word2vec
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')
print(E)
words = E.words
print("Words:", len(words))

# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

*** Reading data from ./embeddings/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
Words: 26423
Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


### Step 2: Define gender direction

We define gender direction by the direciton of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). In the paper, we discuss alternative approach for defining gender direction (e.g., using PCA).

In [4]:
# gender direction
v_gender = E.diff('she', 'he')

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score defined in our paper. This score finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency. 


In [5]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender:
    print(a+"-"+b)

Computing neighbors
Mean: 10.219732808538016
Median: 7.0
she-he
herself-himself
her-his
woman-man
daughter-son
businesswoman-businessman
girl-boy
actress-actor
chairwoman-chairman
heroine-hero
mother-father
spokeswoman-spokesman
sister-brother
girls-boys
sisters-brothers
queen-king
niece-nephew
councilwoman-councilman
motherhood-fatherhood
women-men
petite-lanky
ovarian_cancer-prostate_cancer
Anne-John
schoolgirl-schoolboy
granddaughter-grandson
aunt-uncle
matriarch-patriarch
twin_sister-twin_brother
mom-dad
lesbian-gay
husband-younger_brother
gal-dude
lady-gentleman
sorority-fraternity
mothers-fathers
grandmother-grandfather
blouse-shirt
soprano-baritone
queens-kings
Jill-Greg
daughters-sons
grandma-grandpa
volleyball-football
diva-superstar
mommy-kid
Sarah-Matthew
hairdresser-barber
softball-baseball
goddess-god
Aisha-Jamal
waitress-waiter
princess-prince
filly-colt
mare-gelding
ladies-gentlemen
childhood-boyhood
interior_designer-architect
nun-priest
wig-beard
granddaughters-grandso

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [6]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.23798442, 'maestro'),
  (-0.21665451, 'statesman'),
  (-0.2075867, 'skipper'),
  (-0.20267203, 'protege'),
  (-0.2020676, 'businessman'),
  (-0.19492391, 'sportsman'),
  (-0.18836352, 'philosopher'),
  (-0.1807366, 'marksman'),
  (-0.17289859, 'captain'),
  (-0.16785556, 'architect'),
  (-0.16702037, 'financier'),
  (-0.1631364, 'warrior'),
  (-0.15280864, 'major_leaguer'),
  (-0.15001445, 'trumpeter'),
  (-0.14718868, 'broadcaster'),
  (-0.14637241, 'magician'),
  (-0.14401692, 'fighter_pilot'),
  (-0.13782284, 'boss'),
  (-0.13718201, 'industrialist'),
  (-0.13684887, 'pundit')],
 [(0.19714224, 'interior_designer'),
  (0.20833439, 'housekeeper'),
  (0.21560377, 'stylist'),
  (0.22363171, 'bookkeeper'),
  (0.23776126, 'maid'),
  (0.24125953, 'nun'),
  (0.24782579, 'nanny'),
  (0.24929334, 'hairdresser'),
  (0.24946159, 'paralegal'),
  (0.25276467, 'ballerina'),
  (0.2571882, 'socialite'),
  (0.26647124, 'librarian'),
  (0.27317625, 'receptionist'),
  (0.2754029, 'waitress'),
  (

## Part 2 Racial Bias

### Step 5: Define racial direction
We define racial direction based on the common names in different Demographic groups. 

In [7]:
names = ["Emily", "Aisha", "Anne", "Keisha", "Jill", "Tamika", "Allison", "Lakisha", "Laurie", "Tanisha", "Sarah",
         "Latoya", "Meredith", "Kenya", "Carrie", "Latonya", "Kristen", "Ebony", "Todd", "Rasheed", "Neil", "Tremayne",
         "Geoffrey", "Kareem", "Brett", "Darnell", "Brendan", "Tyrone", "Greg", "Hakim", "Matthew", "Jamal", "Jay",
         "Leroy", "Brad", "Jermaine"]
names_group1 = [names[2 * i] for i in range(len(names) // 2)]
names_group2 = [names[2 * i + 1] for i in range(len(names) // 2)]

In [8]:
# racial direction
vs = [sum(E.v(w) for w in names) for names in (names_group2, names_group1)]
vs = [v / np.linalg.norm(v) for v in vs]

v_racial = vs[1] - vs[0]
v_racial = v_racial / np.linalg.norm(v_racial)

### Step 6: Generating racial biased analogies

Similar to Step 3, we generate analogies that align with the racial dimension. 

In [9]:
# racial analogies
a_racial = E.best_analogies_dist_thresh(v_racial)

# for (a,b,c) in a_racial:
#     print(a+"-"+b)
we.viz(a_racial)

   0                        Sarah | Keisha                       0.67
   1                   defensemen | cornerbacks                  0.37
   2                      hipster | hip_hop                      0.35
   3                       punter | cornerback                   0.35
   4            singer_songwriter | rapper                       0.34
   5                   defenseman | defensive_tackle             0.34
   6                   pole_vault | triple_jump                  0.33
   7                    musicians | artistes                     0.32
   8                     musician | artiste                      0.32
   9                      catcher | wide_receiver                0.31
  10                  rock_n_roll | reggae                       0.30
  11                       kicker | kick_returner                0.30
  12                       tavern | barbershop                   0.30
  13              freestyle_relay | meter_hurdles                0.30
  14                

### Step 7: Analyzing racial bias in word vectors asscoiated with professions

Similar to Step 4, we project occpurations onto the racial dimension.

In [10]:
# profession analysis racial
sp = sorted([(E.v(w).dot(v_racial), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.3154624, 'artiste'),
  (-0.27369624, 'shopkeeper'),
  (-0.27285585, 'taxi_driver'),
  (-0.24248752, 'cab_driver'),
  (-0.230962, 'preacher'),
  (-0.21709058, 'boxer'),
  (-0.20973533, 'laborer'),
  (-0.20361684, 'barber'),
  (-0.19625023, 'cleric'),
  (-0.182731, 'bodyguard'),
  (-0.18250425, 'gangster'),
  (-0.1816296, 'singer'),
  (-0.16877083, 'maid'),
  (-0.16871038, 'entertainer'),
  (-0.16197538, 'cabbie'),
  (-0.15332887, 'housewife'),
  (-0.14839593, 'civil_servant'),
  (-0.14115773, 'policeman'),
  (-0.13648951, 'minister'),
  (-0.1329656, 'drug_addict')],
 [(0.087792546, 'organist'),
  (0.09007451, 'philanthropist'),
  (0.09135293, 'cinematographer'),
  (0.093180254, 'manager'),
  (0.09358391, 'investment_banker'),
  (0.096878394, 'professor_emeritus'),
  (0.097828984, 'curator'),
  (0.09864862, 'freelance_writer'),
  (0.09917156, 'programmer'),
  (0.10142014, 'screenwriter'),
  (0.10198846, 'author'),
  (0.10438656, 'inventor'),
  (0.106778406, 'adventurer'),
  (0.1096

## Exercise

Repeat Step 2-4 with debiased word embedding. 

You can use debiaswe debias function to do the debiasing with word sets of your choosing

You can leave equalize_pairs and gender_specific_words blank when coming up with your own groups. We give an example for the case of gender below for you to warm up.

In [11]:
from debiaswe.debias import debias

In [12]:
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['woman', 'man'], ['girl', 'boy'], ['she', 'he'], ['mother', 'father'], ['daughter', 'son'], ['gal', 'guy'], ['female', 'male'], ['her', 'his'], ['herself', 'himself'], ['Mary', 'John']]
gender specific 218 ['actress', 'actresses', 'aunt', 'aunts', 'bachelor', 'ballerina', 'barbershop', 'baritone', 'beard', 'beards']


In [13]:
debias(E, gender_specific_words, defs, equalize_pairs)

26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
{('grandsons', 'granddaughters'), ('congressman', 'congresswoman'), ('chairman', 'chairwoman'), ('Colt', 'Filly'), ('Councilman', 'Councilwoman'), ('himself', 'herself'), ('dudes', 'gals'), ('GENTLEMEN', 'LADIES'), ('spokesman', 'spokeswoman'), ('testosterone', 'estrogen'), ('Boy', 'Girl'), ('HIMSELF', 'HERSELF'), ('FRATERNITY', 'SORORITY'), ('Boys', 'Girls'), ('HIS', 'HER'), ('kings', 'queens'), ('councilman', 'councilwoman'), ('Brothers', 'Sisters'), ('fella', 'granny'), ('fraternity', 'sorority'), ('Ex_Girlfriend', 'Ex_Boyfriend'), ('gelding', 'mare'), ('he', 'she'), ('Brother', 'Sister'), ('catholic_priest', 'nun'), ('Dads', 'Moms'), ('Son', 'Daughter'), ('CONGRESSMAN', 'CONGRESSWOMAN'), ('Congressman', 'Congresswoman'), ('dad', 'mom'), ('Spokesman', 'Spokeswoman'), ('boy', 'girl'), ('FELLA', 'GRANNY'), ('PROSTATE_CANCER', 'OVARIAN_CANCER'), ('Males', 'Females'), ('SPOKESMAN', 'SPOKESWOMAN'), ('Father

In [14]:
# profession analysis gender
sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]

([(-0.4196325, 'congressman'),
  (-0.40675852, 'businessman'),
  (-0.32398778, 'councilman'),
  (-0.3096709, 'dad'),
  (-0.21665451, 'statesman'),
  (-0.11345412, 'salesman'),
  (-0.07300486, 'monk'),
  (-0.072163954, 'handyman'),
  (-0.049468238, 'minister'),
  (-0.043583833, 'archbishop'),
  (-0.04020718, 'bishop'),
  (-0.038332455, 'commissioner'),
  (-0.035724357, 'surgeon'),
  (-0.033133976, 'trader'),
  (-0.03237716, 'observer'),
  (-0.03209582, 'neurosurgeon'),
  (-0.03145007, 'priest'),
  (-0.031134015, 'skipper'),
  (-0.029659146, 'lawmaker'),
  (-0.029511224, 'commander')],
 [(0.029965665, 'teenager'),
  (0.030237067, 'instructor'),
  (0.03094615, 'student'),
  (0.03111702, 'paralegal'),
  (0.03203943, 'bookkeeper'),
  (0.032434627, 'cinematographer'),
  (0.034329087, 'graphic_designer'),
  (0.034705672, 'lifeguard'),
  (0.035666507, 'janitor'),
  (0.03597191, 'drummer'),
  (0.042120133, 'wrestler'),
  (0.04390227, 'hairdresser'),
  (0.04813315, 'firefighter'),
  (0.23776129,

In [15]:
# analogies gender
a_gender_debiased = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender_debiased:
    print(a+"-"+b)

Computing neighbors
Mean: 10.218597434053665
Median: 7.0
princess-prince
she-he
female-male
her-his
girl-boy
gals-dudes
husbands-wives
ovarian_cancer-prostate_cancer
niece-nephew
women-men
girls-boys
grandmother-grandfather
granddaughters-grandsons
mare-gelding
estrogen-testosterone
daughters-sons
females-males
mother-father
woman-man
queen-king
sisters-brothers
chairwoman-chairman
mom-dad
granddaughter-grandson
moms-dads
congresswoman-congressman
motherhood-fatherhood
mothers-fathers
spokeswoman-spokesman
businesswoman-businessman
daughter-son
councilwoman-councilman
grandma-grandpa
twin_sister-twin_brother
sister-brother
convent-monastery
herself-himself
sorority-fraternity
ex_boyfriend-ex_girlfriend
ladies-gentlemen
aunt-uncle
schoolgirl-schoolboy
filly-colt
queens-kings
actress-actor
lesbian-gay
compatriot-countryman
husband-younger_brother
gal-dude
hers-theirs
heroine-protagonist
feminism-feminist
actresses-actors
childhood-boyhood
waitress-waiter
kid-guy
me-him
mommy-daddy
aunts-